In [1]:
LANG = 'es' # or 'es'
TASK = 'gender' # or 'bot_human'

In [2]:
# Parameters
LANG = "es"
TASK = "bot_human"


In [3]:
import os
from fastai.text import * 

In [4]:
torch.cuda.device_count()

1

In [5]:
proj_path = Path(os.getcwd()).parents[1]

In [6]:
data_path = proj_path/f'clean_data_csv_{LANG}'

In [7]:
df = pd.read_csv(data_path/f'all_{TASK}_data.csv')

In [8]:
df.head()

,bot_human,text
0,bot,Cinco vicios al volante que comprometen la seg...
1,bot,HdC Viñetas: “El profe de prácticas” http://t....
2,human,RT @ISosprisiones: @PP_Asturias @pablocasado_ ...
3,bot,RT @hacerfotos: #Fotografia creativa y artísti...
4,bot,📄📣 Oferta de #empleo de 3 ingeniera/o eléctric...


In [9]:
data_lm = TextLMDataBunch.from_csv(data_path, f'all_{TASK}_data.csv')

In [10]:
data_clas = TextClasDataBunch.from_csv(data_path, f'all_{TASK}_data.csv', 
                                       vocab=data_lm.train_ds.vocab, bs=32)

In [11]:
data_lm.save(f'data_lm_export_{TASK}_{LANG}.pkl')
data_clas.save(f'data_clas_export_{TASK}_{LANG}.pkl')

In [12]:
data_lm = load_data(data_path, fname=f'data_lm_export_{TASK}_{LANG}.pkl')
data_clas = load_data(data_path, fname=f'data_clas_export_{TASK}_{LANG}.pkl', bs=16)

In [13]:
if LANG == 'es':
    weights_pretrained = 'model-30k-vocab-noqrnn'
    itos_pretrained = 'itos_pretrained'
    pretained_data = (weights_pretrained, itos_pretrained)
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5, pretrained_fnames=pretained_data)
else:
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,3.933308,3.698270,0.402839,03:35
2,3.656673,3.434427,0.424751,03:36
3,3.559795,3.379679,0.430466,03:36
4,3.487991,3.341999,0.436313,03:37
5,3.411531,3.308774,0.440130,03:37
6,3.337355,3.273177,0.444390,03:36
7,3.278041,3.241804,0.447956,03:36
8,3.204069,3.221429,0.451143,03:36
9,3.111064,3.211484,0.452464,03:36
10,3.067622,3.210755,0.452607,03:36


In [14]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
1,2.950222,3.139000,0.461666,03:59
2,2.913360,3.079637,0.469841,04:00
3,2.868701,3.039320,0.476849,04:00
4,2.794501,3.014459,0.481849,04:00
5,2.733150,2.996200,0.486163,04:00
6,2.660608,2.987066,0.489013,04:00
7,2.607962,2.984985,0.490976,04:00
8,2.530773,2.986335,0.492334,03:59
9,2.490481,2.987422,0.492801,04:01
10,2.478832,2.990790,0.492858,04:00


In [15]:
if LANG == 'es':
    test_string = "Nació en 1910"
else:
    test_string = "He was born in 1910"
learn.predict(test_string, n_words=50)

'Nació en 1910 como coño , y quiero su cumpleaños . xxbos RT @barbieharp : De mis sentimientos no se trata de disimular lo que sentimos , absolutamente lo mismo , de lo que están pasando . xxbos RT cueros : Qué es el @2010misterchip 💙 😜 💛'

In [16]:
learn.save_encoder(f'ft_enc_{TASK}_{LANG}')

In [17]:
learn_class = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn_class.load_encoder(f'ft_enc_{TASK}_{LANG}')

In [18]:
data_clas.show_batch()

text,target
xxbos xxup rt @juanmalpr : 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮 🇷 🇮,bot
"xxbos xxmaj caja de xxmaj luz de xxmaj cine xxup led , xxup a4 xxmaj xxunk xxup led xxmaj box xxmaj cambio de xxmaj xxunk , xxmaj caja de xxmaj luz xxmaj cine 30 % reservadas xxrep 5 🌟 ❌ xxmaj precio : xxup eur 22,99 ✅ xxmaj precio de la oferta xxup eur xxunk 📉 xxmaj ahorras : xxup eur 4,60 ( 20%) 🔰 xxmaj enlace : https :",bot
"xxbos xxmaj se xxmaj tragan xxmaj viva a xxmaj amara xxmaj la xxmaj negra xxmaj por xxmaj su xxmaj nuevo xxmaj look , xxmaj dicen xxmaj que xxmaj si xxmaj fue xxmaj un xxmaj suaper xxmaj que xxmaj se xxmaj puso xxmaj en xxmaj la … http : / / t.co / xxunk xxmaj unete @teampgv",bot
xxbos xxup rt xxunk : ¿ xxup donde xxup esta xxup el xxup xxunk xxup mejor xxup jugador xxup de xxup la xxup historia ? ¿ xxup donde xxup esta xxup el xxup the xxup best ? ¿ xxup donde xxup esta xxup el xxup xxunk xxup de xxup oro ?,human
xxbos xxmaj mozart xxmaj la xxmaj para ft xxmaj shadow xxmaj blow – xxmaj tu xxmaj con xxmaj el y xxmaj yo xxmaj con xxmaj ella ( xxmaj audio + xxmaj letras ) http : / / t.co / xxunk xxmaj unete @teampgv http : / / t.co / xxunk,bot


In [19]:
learn_class.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,0.435762,0.288846,0.884435,03:52
2,0.365280,0.322353,0.867186,04:04
3,0.421884,0.297878,0.876419,03:55
4,0.433460,0.292421,0.881252,03:58
5,0.405059,0.309969,0.874685,03:55
6,0.436504,0.285140,0.887819,03:58
7,0.418633,0.275719,0.888135,03:56
8,0.380485,0.257837,0.894518,03:55
9,0.361519,0.255442,0.895602,03:56
10,0.364902,0.250042,0.898318,03:56


In [20]:
learn.save(f'model_stage_1_{TASK}_{LANG}')